In [4]:
import pandas as pd
import sqlite3
import datetime
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

from sqlalchemy import create_engine

In [5]:
#connection to a PostgreSQL local database
# ===========================================

# user_name_var = input('What is the username of your database? ')
host_var = input('What is the host address of your database?')
pw = input("Enter the database password: ")
# db_name = input("Enter the name of the database: ")

string_output = "postgresql://postgres"+':'+pw+'@'+host_var+":5432"+'/earthquake_db'
engine = create_engine(string_output)
connection = engine.connect()
print("Connected to database...")

Connected to database...


In [6]:
#create and connect to a sqlite3 db
conn = sqlite3.connect(r'/Users/rosaicelaroman/Desktop/portfolio_work/earthquake_modeling/Database/db/earthquake_db')

Adding 1 month of data

In [7]:
page_url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"

df = pd.read_csv(page_url)
df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-02-06T19:54:55.660Z,19.413666,-155.274338,2.24,1.70,ml,14.0,54.0,NaN,0.16,...,2022-02-06T20:00:26.870Z,"5 km SW of Volcano, Hawaii",earthquake,0.29,0.35,2.490000,8.0,automatic,hv,hv
1,2022-02-06T19:45:01.065Z,60.022300,-147.181100,0.00,2.90,ml,NaN,NaN,NaN,0.65,...,2022-02-06T19:57:35.100Z,"46 km E of Chenega, Alaska",earthquake,NaN,0.30,NaN,NaN,automatic,ak,ak
2,2022-02-06T19:34:55.321Z,63.167000,-150.483300,106.10,2.00,ml,NaN,NaN,NaN,0.42,...,2022-02-06T19:47:33.473Z,"74 km SE of Denali National Park, Alaska",earthquake,NaN,1.10,NaN,NaN,automatic,ak,ak
3,2022-02-06T19:27:45.060Z,38.851833,-122.789001,1.44,0.86,md,10.0,177.0,0.007037,0.02,...,2022-02-06T19:52:13.430Z,"7km WNW of Cobb, CA",earthquake,0.40,0.68,NaN,1.0,automatic,nc,nc
4,2022-02-06T19:27:32.848Z,62.015500,-150.530100,49.00,1.80,ml,NaN,NaN,NaN,0.44,...,2022-02-06T19:40:12.284Z,"36 km SSW of Trapper Creek, Alaska",earthquake,NaN,1.10,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10256,2022-01-07T20:32:10.944Z,60.011100,-148.479300,10.30,2.00,ml,NaN,NaN,NaN,0.30,...,2022-02-01T23:02:58.793Z,"26 km WSW of Chenega, Alaska",earthquake,NaN,0.50,NaN,NaN,reviewed,ak,ak
10257,2022-01-07T20:30:37.581Z,57.714600,-155.765500,3.60,1.10,ml,NaN,NaN,NaN,0.57,...,2022-02-01T23:02:58.585Z,"79 km W of Karluk, Alaska",earthquake,NaN,0.60,NaN,NaN,reviewed,ak,ak
10258,2022-01-07T20:29:18.910Z,38.826167,-122.805667,1.33,0.46,md,20.0,42.0,0.004417,0.02,...,2022-01-12T08:32:10.153Z,"7km NW of The Geysers, CA",earthquake,0.16,0.41,0.053000,3.0,reviewed,nc,nc
10259,2022-01-07T20:24:43.614Z,64.980500,-147.361200,0.00,1.20,ml,NaN,NaN,NaN,0.25,...,2022-02-01T23:02:59.043Z,"12 km ENE of Fox, Alaska",explosion,NaN,0.00,NaN,NaN,reviewed,ak,ak


In [8]:
#append the new data from USGS to both databases
#=============
#PostgreSQL
df.to_sql('earthquake_raw', engine, if_exists='append', index = False)
#=============
#SQLite
cur = conn.cursor()
df.to_sql('earthquake_raw', conn, if_exists='append', index=False)

Load from database to update with new rows and remove duplicate rows

In [9]:
#Dataframe from postgresql
model_df = pd.read_sql_table(
    "earthquake_raw",
    con=engine
)
model_df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06T01:42:39.855Z,37.384000,-117.107500,1.10,1.30,ml,8.0,149.46,0.875000,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.10,0.290000,4.0,automatic,nn,nn
1,2022-01-06T01:31:33.698Z,38.181000,-117.818000,7.30,1.40,ml,11.0,154.64,0.119000,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.00,0.460000,5.0,automatic,nn,nn
2,2022-01-06T01:24:20.083Z,61.340700,-149.885200,30.00,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
3,2022-01-06T01:15:15.851Z,62.870300,-150.762500,111.30,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.80,NaN,NaN,automatic,ak,ak
4,2022-01-06T01:10:44.629Z,64.014500,-148.765700,1.90,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25615,2022-01-07T20:32:10.944Z,60.011100,-148.479300,10.30,2.00,ml,NaN,NaN,NaN,0.3000,...,2022-02-01T23:02:58.793Z,"26 km WSW of Chenega, Alaska",earthquake,NaN,0.50,NaN,NaN,reviewed,ak,ak
25616,2022-01-07T20:30:37.581Z,57.714600,-155.765500,3.60,1.10,ml,NaN,NaN,NaN,0.5700,...,2022-02-01T23:02:58.585Z,"79 km W of Karluk, Alaska",earthquake,NaN,0.60,NaN,NaN,reviewed,ak,ak
25617,2022-01-07T20:29:18.910Z,38.826167,-122.805667,1.33,0.46,md,20.0,42.00,0.004417,0.0200,...,2022-01-12T08:32:10.153Z,"7km NW of The Geysers, CA",earthquake,0.16,0.41,0.053000,3.0,reviewed,nc,nc
25618,2022-01-07T20:24:43.614Z,64.980500,-147.361200,0.00,1.20,ml,NaN,NaN,NaN,0.2500,...,2022-02-01T23:02:59.043Z,"12 km ENE of Fox, Alaska",explosion,NaN,0.00,NaN,NaN,reviewed,ak,ak


In [10]:
#dataframe from SQLite
model_df2 = pd.read_sql_query("SELECT * FROM earthquake_raw", conn)
model_df2

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06T01:42:39.855Z,37.384000,-117.107500,1.10,1.30,ml,8.0,149.46,0.875000,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.10,0.290000,4.0,automatic,nn,nn
1,2022-01-06T01:31:33.698Z,38.181000,-117.818000,7.30,1.40,ml,11.0,154.64,0.119000,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.00,0.460000,5.0,automatic,nn,nn
2,2022-01-06T01:24:20.083Z,61.340700,-149.885200,30.00,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
3,2022-01-06T01:15:15.851Z,62.870300,-150.762500,111.30,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.80,NaN,NaN,automatic,ak,ak
4,2022-01-06T01:10:44.629Z,64.014500,-148.765700,1.90,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20180,2022-01-07T20:32:10.944Z,60.011100,-148.479300,10.30,2.00,ml,NaN,NaN,NaN,0.3000,...,2022-02-01T23:02:58.793Z,"26 km WSW of Chenega, Alaska",earthquake,NaN,0.50,NaN,NaN,reviewed,ak,ak
20181,2022-01-07T20:30:37.581Z,57.714600,-155.765500,3.60,1.10,ml,NaN,NaN,NaN,0.5700,...,2022-02-01T23:02:58.585Z,"79 km W of Karluk, Alaska",earthquake,NaN,0.60,NaN,NaN,reviewed,ak,ak
20182,2022-01-07T20:29:18.910Z,38.826167,-122.805667,1.33,0.46,md,20.0,42.00,0.004417,0.0200,...,2022-01-12T08:32:10.153Z,"7km NW of The Geysers, CA",earthquake,0.16,0.41,0.053000,3.0,reviewed,nc,nc
20183,2022-01-07T20:24:43.614Z,64.980500,-147.361200,0.00,1.20,ml,NaN,NaN,NaN,0.2500,...,2022-02-01T23:02:59.043Z,"12 km ENE of Fox, Alaska",explosion,NaN,0.00,NaN,NaN,reviewed,ak,ak


In [11]:
#check for duplicates in postgres data
boolean = model_df.duplicated(subset=['id']).any()
print(boolean)

True


In [12]:
#check for duplicates in sqlite data
boolean = model_df2.duplicated(subset=['id']).any()
print(boolean)

False


In [13]:
#drop duplicates in postgres data
model_df.drop_duplicates(subset=['id'], inplace=True)
print(model_df)

                           time  latitude  longitude  depth  mag magType  \
0      2022-01-06T01:42:39.855Z   37.3840  -117.1075    1.1  1.3      ml   
1      2022-01-06T01:31:33.698Z   38.1810  -117.8180    7.3  1.4      ml   
2      2022-01-06T01:24:20.083Z   61.3407  -149.8852   30.0  1.5      ml   
3      2022-01-06T01:15:15.851Z   62.8703  -150.7625  111.3  2.1      ml   
4      2022-01-06T01:10:44.629Z   64.0145  -148.7657    1.9  1.5      ml   
...                         ...       ...        ...    ...  ...     ...   
25581  2022-01-07T22:29:38.096Z   -3.6943    68.4041   10.0  4.7      mb   
25601  2022-01-07T21:29:14.733Z   28.3243   142.9234   10.0  4.7      mb   
25608  2022-01-07T20:45:24.409Z   37.8723   101.1333   10.0  4.2      mb   
25613  2022-01-07T20:33:36.548Z   62.9584  -148.0912   67.6  1.3      ml   
25616  2022-01-07T20:30:37.581Z   57.7146  -155.7655    3.6  1.1      ml   

        nst     gap    dmin     rms  ...                   updated  \
0       8.0  149.

In [14]:
#drop duplicates in sqlite data
model_df2.drop_duplicates(subset=['id'], inplace=True)
print(model_df2)

                           time   latitude   longitude   depth   mag magType  \
0      2022-01-06T01:42:39.855Z  37.384000 -117.107500    1.10  1.30      ml   
1      2022-01-06T01:31:33.698Z  38.181000 -117.818000    7.30  1.40      ml   
2      2022-01-06T01:24:20.083Z  61.340700 -149.885200   30.00  1.50      ml   
3      2022-01-06T01:15:15.851Z  62.870300 -150.762500  111.30  2.10      ml   
4      2022-01-06T01:10:44.629Z  64.014500 -148.765700    1.90  1.50      ml   
...                         ...        ...         ...     ...   ...     ...   
20180  2022-01-07T20:32:10.944Z  60.011100 -148.479300   10.30  2.00      ml   
20181  2022-01-07T20:30:37.581Z  57.714600 -155.765500    3.60  1.10      ml   
20182  2022-01-07T20:29:18.910Z  38.826167 -122.805667    1.33  0.46      md   
20183  2022-01-07T20:24:43.614Z  64.980500 -147.361200    0.00  1.20      ml   
20184  2022-01-07T20:23:20.530Z  19.406667 -155.268000   10.04  0.98      md   

        nst     gap      dmin     rms  

In [15]:
#replace the table in postgres with one without duplicates
model_df.to_sql('earthquake_raw', engine, if_exists='replace', index = False)

In [16]:
#replace the table in sqlite with one without duplicates
model_df2.to_sql('earthquake_raw', conn, if_exists='replace', index = False)

In [17]:
#copy data as csv as backup method
model_df.to_csv("earthquake_raw.csv",index=False)

In [18]:
# Close connection to SQLite database
conn.close()
connection.close()